In [13]:
import urllib.request
import pprint as pp
import datetime, pymysql, json
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
from models import *
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, and_
%matplotlib inline
import numpy as np
import connections as cnnt


In [ ]:
curs, engine, session = cnnt.cursor()
data=open("config.json").read()
json_data = json.loads(data)

user = json_data["users"][0]["user"]
password = json_data["users"][0]["password"]
host = json_data["users"][0]["host"]
port = json_data["users"][0]["port"]
DB = json_data["users"][0]["DB"]
kobis_key = json_data["users"][0]["kobis_key"]

In [30]:
#인섵하는 함수. 헬프함수
def insert_into_db(movie_table):
    session = cnnt.cursor()[2]
    movie_table = pd.DataFrame(movie_table).fillna(-1)
    for movie_name, section_audi in zip(movie_table.index, movie_table.values):
        for search_date,today_audi in zip(movie_table.keys(), section_audi):
            search_date = datetime.datetime.strptime(search_date,'%Y%M%d').date()
            if today_audi != -1 : 
                add_movie_info = KobisMovieInfo(movie_name, search_date, today_audi)
                session.merge(add_movie_info)
    session.commit()
    session.close()
            
    return "Inserted Succesfully"

In [31]:
#영화명, 관객수, 조회일 크롤링해온다음 디비에 저장시킴, 저장시킨걸 데이터프레임으로 보여줌
def get_movie_audiance_num_per_date(input_date, time_section):
    #params : week or month
    input_date = datetime.datetime.strptime(input_date,'%Y-%m-%d')
    searching_date = []

    if not input_date:
        print("오늘날짜 업데이트가 되지 않았어요ㅜㅜ 어제날짜로 검색하시겠습니까?")
    
    while time_section:
        searching_date.append(input_date.strftime("%Y%m%d"))
        input_date -= datetime.timedelta(days=1)
        time_section -= 1


    movie_info_per_date = {}
    for date_time in searching_date:
        base_url = urllib.request.urlopen(f"http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key={kobis_key}&targetDt={date_time}")
        sources = json.load(base_url)
        movie_infos = sources["boxOfficeResult"]['dailyBoxOfficeList']
    
        audiacc_per_movie = {}
        for movie_info in movie_infos:
            audiacc_per_movie[movie_info['movieNm']] = movie_info['audiCnt']
        movie_info_per_date[date_time] = audiacc_per_movie


    movie_table = pd.DataFrame(movie_info_per_date)
    movie_table["sum"] = movie_table.sum(axis = 1)
    movie_table = movie_table.sort_values(by = "sum", ascending= False)[:5]
    del movie_table["sum"]
    
    insert_into_db(movie_table)
    return movie_table

In [33]:
#디비에있는것 그래프로 그려줌
def query_and_draw(start_date, end_date):
    session = cnnt.cursor()[2]
    start_date = datetime.datetime.strptime(start_date,'%Y-%M-%d').date()
    end_date = datetime.datetime.strptime(end_date,'%Y-%M-%d').date()
    query = pd.DataFrame(session.query(KobisMovieInfo.movie_name, KobisMovieInfo.search_date,KobisMovieInfo.today_audi).filter(start_date >= KobisMovieInfo.search_date).filter(KobisMovieInfo.search_date >= end_date).all())
    movie_table =  pd.DataFrame(index = list(set(query.movie_name)), columns = sorted(list(set(query.search_date))))
    
    for i in query.values:
        movie_table.loc[movie_table.index == i[0] , movie_table.columns == i[1]] = i[2]
    movie_table["sum"] = movie_table.sum(axis = 1)
    movie_table = movie_table.sort_values(by = "sum", ascending= False)[:5]
    del movie_table["sum"]
    del query
    
    movie_table.columns = movie_table.columns.map(lambda x : x.strftime('%Y%m%d'))
    movie_name = list(movie_table.index)
    date_time = movie_table.columns
    auc_rate = movie_table.values

    plt.figure(figsize = (15,12))
    for i in auc_rate:
        plt.plot(date_time,i, marker = "o")
    plt.xlabel("Date")
    plt.ylabel("AudiAcc")
    plt.autoscale(enable = True, axis = "y")
    
    
    font_name = fm.FontProperties(fname = "./font/NanumGothic.ttf").get_name()
    plt.rc('font', family = font_name)
    plt.legend(movie_name, loc = "best")
    session.close()
    return plt.show()
    

AttributeError: 'DataFrame' object has no attribute 'movie_name'

In [42]:
def show_movie_trend_graph(movie_table):
    
    movie_name = list(movie_table.index)
    date_time = movie_table.columns
    auc_rate = movie_table.values

    plt.figure(figsize = (30,20))
    for i in auc_rate:
        plt.plot(date_time,i, marker = "o")
    plt.xlabel("Date")
    plt.ylabel("AudiAcc")
    plt.autoscale(enable = True, axis = "y")
    
    font_name = fm.FontProperties(fname = "./font/NanumGothic.ttf").get_name()
    plt.rc('font', family = font_name)
    plt.legend(movie_name, loc = "best")
    return plt.show()

In [43]:
query_and_draw('2018-01-30','2018-01-15')

AttributeError: 'DataFrame' object has no attribute 'movie_name'

In [44]:
movie_table = get_movie_audiance_num_per_date('2018-01-30',16)
# insert_into_db(movie_table)
show_movie_trend_graph(movie_table)

InvalidRequestError: One or more mappers failed to initialize - can't proceed with initialization of other mappers. Triggering mapper: 'Mapper|ActorsOfMovie|ActorsOfMovie'. Original exception was: Error creating backref 'DirectorOfMovie' on relationship 'ActorsOfMovie.moviecode_connetion': property of that name exists on mapper 'Mapper|BaseMovieInfo|BaseMovieInfo'

In [ ]:
movie_table

In [ ]:
query_and_draw()

In [ ]:
# movie_table

In [ ]:
# datetime.datetime.today() - datetime.datetime.strftime("2018-06-05","%Y%m%d")
# (datetime.datetime.today()- datetime.datetime.strptime("2018-06-05",'%Y-%m-%d'))

In [ ]:
day = 7
today = datetime.datetime.today() - datetime.timedelta(days=2)
searching_date = []
while day:
    day -= 1
    today -= datetime.timedelta(days=1)
    searching_date.append(today.strftime("%Y%m%d"))


movie_info_per_date = {}
for date_time in searching_date:
    base_url = urllib.request.urlopen(f"http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=430156241533f1d058c603178cc3ca0e&targetDt={date_time}")
    sources = json.load(base_url)
    movie_infos = sources["boxOfficeResult"]['dailyBoxOfficeList']
    
    audiacc_per_movie = {}
    for movie_info in movie_infos:
        audiacc_per_movie[movie_info['movieNm']] = int(movie_info['audiCnt'])
    movie_info_per_date[date_time] = audiacc_per_movie


week_table = pd.DataFrame(movie_info_per_date).fillna(0)
week_table["sum"] = week_table.sum(axis = 1) 
week_table = week_table.sort_values(by = "sum", ascending= False)[:5]
del week_table["sum"]

movie_name = list(week_table.index)
date_time = week_table.columns
auc_rate = week_table.values

week_table

In [ ]:
movie_trend_graph("week")